# Camada Bronze — Rio de Janeiro

### Objetivo  
Ingerir os arquivos brutos do **Inside Airbnb – Rio de Janeiro** diretamente para o formato Delta, sem qualquer transformação.  
A camada Bronze preserva:

- estrutura original dos arquivos,
- nomes e tipos de coluna,
- granularidade completa,
- fidelidade total à fonte.

Essas tabelas servem como base imutável para as próximas camadas (Silver e Gold).


In [0]:
%sql
CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.bronze_listings_rio AS
SELECT *
FROM read_files(
  '/Volumes/mvp_airbnb_catalog/default/airbnb_volume/rio/bronze/listings.csv',
  format      => 'csv',
  header      => true,
  inferSchema => true,
  multiLine   => true,
  quote       => '"',
  escape      => '"'
);

CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.bronze_calendar_rio AS
SELECT *
FROM read_files(
  '/Volumes/mvp_airbnb_catalog/default/airbnb_volume/rio/bronze/calendar.csv',
  format      => 'csv',
  header      => true,
  inferSchema => true,
  multiLine   => true,
  quote       => '"',
  escape      => '"'
);

CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.bronze_reviews_rio AS
SELECT *
FROM read_files(
  '/Volumes/mvp_airbnb_catalog/default/airbnb_volume/rio/bronze/reviews.csv',
  format      => 'csv',
  header      => true,
  inferSchema => true,
  multiLine   => true,
  quote       => '"',
  escape      => '"'
);

CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.bronze_neighbourhoods_rio AS
SELECT *
FROM read_files(
  '/Volumes/mvp_airbnb_catalog/default/airbnb_volume/rio/bronze/neighbourhoods_rio.csv',
  format      => 'csv',
  header      => true,
  inferSchema => true,
  multiLine   => true,
  quote       => '"',
  escape      => '"'
);



In [0]:
%sql
-- Evidencia tabela bronze_listings_rio

SELECT * FROM mvp_airbnb_catalog.default.bronze_listings_rio
LIMIT 5

In [0]:
%sql
-- Evidencia tabela bronze_calendar_rio

SELECT * FROM mvp_airbnb_catalog.default.bronze_calendar_rio
LIMIT 5

In [0]:
%sql
-- Evidencia tabela bronze_reviews_rio

SELECT * FROM mvp_airbnb_catalog.default.bronze_reviews_rio
LIMIT 5

In [0]:
%sql
-- Evidencia tabela bronze_neighbourhoods_rio

SELECT * FROM mvp_airbnb_catalog.default.bronze_neighbourhoods_rio
LIMIT 5

# Camada Bronze — New York City

### Objetivo  
Realizar a ingestão bruta dos datasets do **Inside Airbnb – NYC**, garantindo que os dados sejam armazenados exatamente como recebidos.  
A camada Bronze assegura:

- reprodutibilidade da ingestão,
- ausência de limpeza ou padronização prematura,
- persistência íntegra dos dados originais.

Essa camada forma o ponto de partida seguro do pipeline ETL.


In [0]:
%sql
CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.bronze_listings_nyc AS
SELECT *
FROM read_files(
  '/Volumes/mvp_airbnb_catalog/default/airbnb_volume/nyc/analysis-only/listings-nyc.csv',
  format      => 'csv',
  header      => true,
  inferSchema => true,
  multiLine   => true,
  quote       => '"',
  escape      => '"'
);

CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.bronze_calendar_nyc AS
SELECT *
FROM read_files(
  '/Volumes/mvp_airbnb_catalog/default/airbnb_volume/nyc/analysis-only/calendar-nyc.csv',
  format      => 'csv',
  header      => true,
  inferSchema => true,
  multiLine   => true,
  quote       => '"',
  escape      => '"'
);

CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.bronze_reviews_nyc AS
SELECT *
FROM read_files(
  '/Volumes/mvp_airbnb_catalog/default/airbnb_volume/nyc/analysis-only/reviews-nyc.csv',
  format      => 'csv',
  header      => true,
  inferSchema => true,
  multiLine   => true,
  quote       => '"',
  escape      => '"'
);
CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.bronze_neighbourhoods_nyc AS
SELECT *
FROM read_files(
  '/Volumes/mvp_airbnb_catalog/default/airbnb_volume/nyc/analysis-only/neighbourhoods_nyc.csv',
  format      => 'csv',
  header      => true,
  inferSchema => true,
  multiLine   => true,
  quote       => '"',
  escape      => '"'
);



In [0]:
%sql
-- Evidencia tabela bronze_listings_nyc

SELECT * FROM mvp_airbnb_catalog.default.bronze_listings_nyc
LIMIT 5

In [0]:
%sql
-- Evidencia tabela bronze_calendar_nyc

SELECT * FROM mvp_airbnb_catalog.default.bronze_calendar_nyc
LIMIT 5

In [0]:
%sql
-- Evidencia tabela bronze_reviews_nyc

SELECT * FROM mvp_airbnb_catalog.default.bronze_reviews_nyc
LIMIT 5

In [0]:
%sql
-- Evidencia tabela bronze_neighbourhoods_nyc

SELECT * FROM mvp_airbnb_catalog.default.bronze_neighbourhoods_nyc
LIMIT 5

#  Camada Silver — Rio de Janeiro

### Objetivo  
Aplicar limpeza, padronização e enriquecimento sem alterar a granularidade dos dados.  
Principais ações:

- conversão de tipos numéricos e booleanos,
- padronização de colunas (`host_type`, `room_type`, etc.),
- criação de campos derivados,
- tratamento de nulos e normalização de valores,
- harmonização das tabelas para o modelo Gold.

A camada Silver deixa os dados consistentes e analisáveis.


In [0]:
%sql

---------- silver_calendar_rio ----------
CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.silver_listings_rio AS
SELECT
  -- Mantém TODAS as colunas originais da Bronze
  b.*,

  -- Preço numérico (remove símbolos, espaços e vírgulas)
  CAST(
    NULLIF(
      regexp_replace(b.price, '[^0-9\\.]', ''),  -- mantém só dígitos e ponto
      ''
    ) AS DOUBLE
  ) AS price_numeric,

  -- Flags booleanas a partir de strings ('t','f','true','false','yes','no', etc.)
  CASE
    WHEN lower(b.host_is_superhost) IN ('t','true','1','yes','y') THEN true
    WHEN lower(b.host_is_superhost) IN ('f','false','0','no','n') THEN false
    ELSE NULL
  END AS host_is_superhost_bool,

  CASE
    WHEN lower(b.host_identity_verified) IN ('t','true','1','yes','y') THEN true
    WHEN lower(b.host_identity_verified) IN ('f','false','0','no','n') THEN false
    ELSE NULL
  END AS host_identity_verified_bool,

  CASE
    WHEN lower(b.instant_bookable) IN ('t','true','1','yes','y') THEN true
    WHEN lower(b.instant_bookable) IN ('f','false','0','no','n') THEN false
    ELSE NULL
  END AS instant_bookable_bool,

  -- Datas e recência de reviews
  TO_DATE(b.last_review) AS last_review_date,
  DATEDIFF(CURRENT_DATE(), TO_DATE(b.last_review)) AS review_recency_days,

  -- Flags de tipo de acomodação
  CASE WHEN b.room_type = 'Entire home/apt' THEN true ELSE false END AS is_entire_home,
  CASE WHEN b.room_type = 'Private room'    THEN true ELSE false END AS is_private_room,
  CASE WHEN b.room_type = 'Shared room'     THEN true ELSE false END AS is_shared_room,
  CASE WHEN b.room_type = 'Hotel room'      THEN true ELSE false END AS is_hotel_room,

  -- Limpeza de texto: description
  regexp_replace(
    regexp_replace(
      regexp_replace(
        b.description,
        '(?i)<br\\s*/?>', ' '        -- remove <br>, <br/>, <BR>, etc.
      ),
      '[\\r\\n\\t]+', ' '            -- remove quebras de linha e tabs
    ),
    '\\s+', ' '                      -- normaliza múltiplos espaços
  ) AS description_clean,

  -- Limpeza de texto: neighborhood_overview (se existir no schema)
  regexp_replace(
    regexp_replace(
      regexp_replace(
        b.neighborhood_overview,
        '(?i)<br\\s*/?>', ' '
      ),
      '[\\r\\n\\t]+', ' '
    ),
    '\\s+', ' '
  ) AS neighborhood_overview_clean,

  -- Limpeza de texto: host_about (se existir)
  regexp_replace(
    regexp_replace(
      regexp_replace(
        b.host_about,
        '(?i)<br\\s*/?>', ' '
      ),
      '[\\r\\n\\t]+', ' '
    ),
    '\\s+', ' '
  ) AS host_about_clean

FROM
  mvp_airbnb_catalog.default.bronze_listings_rio AS b;


---------- silver_calendar_rio ----------
CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.silver_calendar_rio AS
SELECT
  -- Mantém todas as colunas originais
  b.*,

  -- Converte a data para tipo DATE
  TO_DATE(b.date) AS date_date,

  -- Converte 'available' para boolean
  CASE
    WHEN lower(b.available) IN ('t','true','1','yes','y') THEN true
    WHEN lower(b.available) IN ('f','false','0','no','n') THEN false
    ELSE NULL
  END AS available_bool,

  -- Cria flag de ocupado (inverso da disponibilidade)
  CASE
    WHEN lower(b.available) IN ('t','true','1','yes','y') THEN false
    WHEN lower(b.available) IN ('f','false','0','no','n') THEN true
    ELSE NULL
  END AS is_occupied,

  -- Preço numérico (se a coluna existir)
  CAST(
    NULLIF(
      regexp_replace(b.price, '[^0-9\\.]', ''),   -- remove tudo exceto dígitos e ponto
      ''
    ) AS DOUBLE
  ) AS price_numeric,

  -- Preço ajustado numérico (se a coluna adjusted_price existir)
  CAST(
    NULLIF(
      regexp_replace(b.adjusted_price, '[^0-9\\.]', ''), 
      ''
    ) AS DOUBLE
  ) AS adjusted_price_numeric,

  -- Receita potencial (pode ser refinado depois)
  CASE
    WHEN lower(b.available) IN ('f','false','0','no','n')
         AND NULLIF(regexp_replace(b.price, '[^0-9\\.]', ''), '') IS NOT NULL
    THEN CAST(regexp_replace(b.price, '[^0-9\\.]', '') AS DOUBLE)
    ELSE 0.0
  END AS revenue_potential

FROM
  mvp_airbnb_catalog.default.bronze_calendar_rio AS b;


---------- silver_reviews_rio ----------

CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.silver_reviews_rio AS
SELECT
  -- mantém tudo que veio do Bronze
  b.*,

  -- data em tipo DATE
  TO_DATE(b.date) AS review_date,

  -- limpeza de texto dos comentários
  regexp_replace(
    regexp_replace(
      regexp_replace(
        b.comments,
        '(?i)<br\\s*/?>', ' '       -- remove <br>, <br/>, etc.
      ),
      '[\\r\\n\\t]+', ' '           -- remove quebras de linha e tabs
    ),
    '\\s+', ' '                     -- normaliza múltiplos espaços
  ) AS comments_clean

FROM
  mvp_airbnb_catalog.default.bronze_reviews_rio AS b;



In [0]:
%sql
-- Evidencia tabela silver_listings_rio
SELECT * FROM mvp_airbnb_catalog.default.silver_listings_rio
LIMIT 5

In [0]:
%sql
-- Evidencia tabela silver_calendar_rio
SELECT * FROM mvp_airbnb_catalog.default.silver_calendar_rio
LIMIT 5

In [0]:
%sql
-- Evidencia tabela silver_reviews_rio
SELECT * FROM mvp_airbnb_catalog.default.silver_reviews_rio
LIMIT 5

# Camada Silver — New York City

### Objetivo  
Executar o mesmo conjunto de transformações aplicadas ao Rio de Janeiro, garantindo consistência entre mercados.  
Inclui:

- normalização de tipos,
- limpeza e padronização de estruturas,
- ajustes semânticos,
- preparação para integração no modelo dimensional.

É a camada que garante uniformidade entre as duas cidades antes da modelagem.



In [0]:
%sql
---------- silver_listings_nyc ----------
CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.silver_listings_nyc AS
SELECT
  -- todas as colunas originais do Bronze
  b.*,

  -- preço numérico (USD)
  CAST(
    NULLIF(
      regexp_replace(b.price, '[^0-9\\.]', ''),  -- só dígitos e ponto
      ''
    ) AS DOUBLE
  ) AS price_numeric,

  -- flags booleanas
  CASE
    WHEN lower(b.host_is_superhost) IN ('t','true','1','yes','y') THEN true
    WHEN lower(b.host_is_superhost) IN ('f','false','0','no','n') THEN false
    ELSE NULL
  END AS host_is_superhost_bool,

  CASE
    WHEN lower(b.host_identity_verified) IN ('t','true','1','yes','y') THEN true
    WHEN lower(b.host_identity_verified) IN ('f','false','0','no','n') THEN false
    ELSE NULL
  END AS host_identity_verified_bool,

  CASE
    WHEN lower(b.instant_bookable) IN ('t','true','1','yes','y') THEN true
    WHEN lower(b.instant_bookable) IN ('f','false','0','no','n') THEN false
    ELSE NULL
  END AS instant_bookable_bool,

  -- datas e recência
  TO_DATE(b.last_review) AS last_review_date,
  DATEDIFF(CURRENT_DATE(), TO_DATE(b.last_review)) AS review_recency_days,

  -- flags de tipo de acomodação
  CASE WHEN b.room_type = 'Entire home/apt' THEN true ELSE false END AS is_entire_home,
  CASE WHEN b.room_type = 'Private room'    THEN true ELSE false END AS is_private_room,
  CASE WHEN b.room_type = 'Shared room'     THEN true ELSE false END AS is_shared_room,
  CASE WHEN b.room_type = 'Hotel room'      THEN true ELSE false END AS is_hotel_room,

  -- limpeza de texto: description
  regexp_replace(
    regexp_replace(
      regexp_replace(
        b.description,
        '(?i)<br\\s*/?>', ' '
      ),
      '[\\r\\n\\t]+', ' '
    ),
    '\\s+', ' '
  ) AS description_clean,

  -- limpeza de texto: neighborhood_overview
  regexp_replace(
    regexp_replace(
      regexp_replace(
        b.neighborhood_overview,
        '(?i)<br\\s*/?>', ' '
      ),
      '[\\r\\n\\t]+', ' '
    ),
    '\\s+', ' '
  ) AS neighborhood_overview_clean,

  -- limpeza de texto: host_about
  regexp_replace(
    regexp_replace(
      regexp_replace(
        b.host_about,
        '(?i)<br\\s*/?>', ' '
      ),
      '[\\r\\n\\t]+', ' '
    ),
    '\\s+', ' '
  ) AS host_about_clean

FROM
  mvp_airbnb_catalog.default.bronze_listings_nyc AS b;


  ---------- silver_calendar_nyc ----------

  CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.silver_calendar_nyc AS
SELECT
  b.*,

  TO_DATE(b.date) AS date_date,

  CASE
    WHEN lower(b.available) IN ('t','true','1','yes','y') THEN true
    WHEN lower(b.available) IN ('f','false','0','no','n') THEN false
    ELSE NULL
  END AS available_bool,

  CASE
    WHEN lower(b.available) IN ('t','true','1','yes','y') THEN false
    WHEN lower(b.available) IN ('f','false','0','no','n') THEN true
    ELSE NULL
  END AS is_occupied,

  CAST(
    NULLIF(
      regexp_replace(b.price, '[^0-9\\.]', ''),
      ''
    ) AS DOUBLE
  ) AS price_numeric,

  CAST(
    NULLIF(
      regexp_replace(b.adjusted_price, '[^0-9\\.]', ''),
      ''
    ) AS DOUBLE
  ) AS adjusted_price_numeric

FROM
  mvp_airbnb_catalog.default.bronze_calendar_nyc AS b;

  ---------- silver_reviews_nyc ----------
CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.silver_reviews_nyc AS
SELECT
  b.*,

  TO_DATE(b.date) AS review_date,

  regexp_replace(
    regexp_replace(
      regexp_replace(
        b.comments,
        '(?i)<br\\s*/?>', ' '
      ),
      '[\\r\\n\\t]+', ' '
    ),
    '\\s+', ' '
  ) AS comments_clean

FROM
  mvp_airbnb_catalog.default.bronze_reviews_nyc AS b;

In [0]:
%sql
-- Evidencia tabela silver_listings_nyc
SELECT * FROM mvp_airbnb_catalog.default.silver_listings_nyc
LIMIT 5

In [0]:
%sql
-- Evidencia tabela silver_calendar_nyc
SELECT * FROM mvp_airbnb_catalog.default.silver_calendar_nyc
LIMIT 5

In [0]:
%sql
-- Evidencia tabela silver_reviews_nyc
SELECT * FROM mvp_airbnb_catalog.default.silver_reviews_nyc
LIMIT 5

# Camada Gold — Modelo Dimensional Final

### Objetivo  
Construir o **modelo dimensional** do MVP, adequado para análise, BI e exploração estatística.  
Inclui:

- criação das **dimensões** (listings, hosts, neighbourhoods),
- criação das **tabelas fato** (calendar, listings, reviews),
- aplicação das regras de negócio,
- garantia da integridade referencial entre fatos e dimensões.

A camada Gold representa o produto final do pipeline, pronto para uso analítico e visualizações.


In [0]:
%sql
-- =====================================================================
-- DIM_NEIGHBOURHOODS  (Gold diretamente a partir da Bronze)
-- =====================================================================

CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.dim_neighbourhoods AS
SELECT
  'Rio de Janeiro'                AS city,
  TRIM(neighbourhood)             AS neighbourhood,
  neighbourhood_group,
  geometry_json
FROM mvp_airbnb_catalog.default.bronze_neighbourhoods_rio

UNION ALL

SELECT
  'New York City'                 AS city,
  TRIM(neighbourhood)             AS neighbourhood,
  neighbourhood_group,
  geometry_json
FROM mvp_airbnb_catalog.default.bronze_neighbourhoods_nyc
;

-- PK: (city, neighbourhood)
ALTER TABLE mvp_airbnb_catalog.default.dim_neighbourhoods
ALTER COLUMN city          SET NOT NULL;

ALTER TABLE mvp_airbnb_catalog.default.dim_neighbourhoods
ALTER COLUMN neighbourhood SET NOT NULL;

ALTER TABLE mvp_airbnb_catalog.default.dim_neighbourhoods
ADD CONSTRAINT pk_dim_neighbourhoods
PRIMARY KEY (city, neighbourhood)
;

-- =====================================================================
-- DIM_HOSTS  (Gold a partir das Silver de listings)
-- =====================================================================

CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.dim_hosts AS
WITH listings_union AS (
    SELECT
      'Rio de Janeiro'  AS city,
      host_id,
      host_name,
      host_since,
      host_is_superhost_bool,
      host_total_listings_count,
      id AS listing_id
    FROM mvp_airbnb_catalog.default.silver_listings_rio

    UNION ALL

    SELECT
      'New York City'   AS city,
      host_id,
      host_name,
      host_since,
      host_is_superhost_bool,
      host_total_listings_count,
      id AS listing_id
    FROM mvp_airbnb_catalog.default.silver_listings_nyc
)
SELECT
  city,
  host_id,
  ANY_VALUE(host_name)               AS host_name,              
  MIN(host_since)                    AS host_since,
  MAX(host_is_superhost_bool)        AS host_is_superhost_bool,
  MAX(host_total_listings_count)     AS host_total_listings_count,
  COUNT(DISTINCT listing_id)         AS calculated_host_listings_count,
  CASE
    WHEN COUNT(DISTINCT listing_id) >= 3 THEN 'professional'
    WHEN COUNT(DISTINCT listing_id) = 2 THEN 'small_host'
    WHEN COUNT(DISTINCT listing_id) = 1 THEN 'individual'
    ELSE 'unknown'
  END                                AS host_type
FROM listings_union
GROUP BY city, host_id
;

-- PK: (city, host_id)
ALTER TABLE mvp_airbnb_catalog.default.dim_hosts
ALTER COLUMN city    SET NOT NULL;

ALTER TABLE mvp_airbnb_catalog.default.dim_hosts
ALTER COLUMN host_id SET NOT NULL;

ALTER TABLE mvp_airbnb_catalog.default.dim_hosts
ADD CONSTRAINT pk_dim_hosts
PRIMARY KEY (city, host_id)
;

-- =====================================================================
-- DIM_LISTINGS  (Gold a partir das Silver de listings)
-- =====================================================================

CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.dim_listings AS
WITH listings_union AS (
    SELECT
      'Rio de Janeiro'  AS city,
      *
    FROM mvp_airbnb_catalog.default.silver_listings_rio

    UNION ALL

    SELECT
      'New York City'   AS city,
      *
    FROM mvp_airbnb_catalog.default.silver_listings_nyc
)
SELECT
  city,
  CAST(id AS BIGINT)                                     AS listing_id,
  host_id,
  TRIM(neighbourhood_cleansed)                          AS neighbourhood_cleansed,
  property_type,
  room_type,
  accommodates,
  bathrooms,
  bedrooms,
  beds,
  price_numeric,
  review_scores_rating,
  reviews_per_month,
  estimated_occupancy_l365d,
  estimated_revenue_l365d,

  -- Flags de tipo de acomodação
  CASE WHEN room_type = 'Entire home/apt' THEN TRUE ELSE FALSE END AS is_entire_home,
  CASE WHEN room_type = 'Private room'    THEN TRUE ELSE FALSE END AS is_private_room,
  CASE WHEN room_type = 'Shared room'     THEN TRUE ELSE FALSE END AS is_shared_room,
  CASE WHEN room_type = 'Hotel room'      THEN TRUE ELSE FALSE END AS is_hotel_room,

  -- Atributos de host replicados
  host_is_superhost_bool,
  host_identity_verified_bool,
  instant_bookable_bool,

  -- Datas / recência de review
  last_review_date,
  review_recency_days,

  -- Preço válido?
  CASE WHEN price_numeric IS NOT NULL THEN TRUE ELSE FALSE END AS has_valid_price,

  -- Moeda
  CASE
    WHEN city = 'Rio de Janeiro' THEN 'BRL'
    WHEN city = 'New York City'  THEN 'USD'
    ELSE 'UNKNOWN'
  END                                                    AS currency
FROM listings_union
;

-- PK: (city, listing_id)
ALTER TABLE mvp_airbnb_catalog.default.dim_listings
ALTER COLUMN city       SET NOT NULL;

ALTER TABLE mvp_airbnb_catalog.default.dim_listings
ALTER COLUMN listing_id SET NOT NULL;

ALTER TABLE mvp_airbnb_catalog.default.dim_listings
ADD CONSTRAINT pk_dim_listings
PRIMARY KEY (city, listing_id)
;

-- FK: dim_listings.(city, host_id) -> dim_hosts
ALTER TABLE mvp_airbnb_catalog.default.dim_listings
ADD CONSTRAINT fk_dim_listings_host
FOREIGN KEY (city, host_id)
REFERENCES mvp_airbnb_catalog.default.dim_hosts (city, host_id)
;

-- FK: dim_listings.(city, neighbourhood_cleansed) -> dim_neighbourhoods
ALTER TABLE mvp_airbnb_catalog.default.dim_listings
ADD CONSTRAINT fk_dim_listings_neighbourhood
FOREIGN KEY (city, neighbourhood_cleansed)
REFERENCES mvp_airbnb_catalog.default.dim_neighbourhoods (city, neighbourhood)
;

-- CHECK: preço não negativo
ALTER TABLE mvp_airbnb_catalog.default.dim_listings
ADD CONSTRAINT chk_dim_listings_price_positive
CHECK (price_numeric IS NULL OR price_numeric >= 0)
;

-- =====================================================================
-- FACT_CALENDAR  (Gold a partir das Silver de calendar + dim_listings)
-- =====================================================================

CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.fact_calendar AS
WITH calendar_union AS (
    -- RIO
    SELECT
      'Rio de Janeiro'                   AS city,
      CAST(listing_id AS BIGINT)         AS listing_id,
      date_date,                         
      available_bool,
      is_occupied,
      price_numeric,
      adjusted_price_numeric,            
      minimum_nights,
      maximum_nights
    FROM mvp_airbnb_catalog.default.silver_calendar_rio

    UNION ALL

    -- NYC
    SELECT
      'New York City'                    AS city,
      CAST(listing_id AS BIGINT)         AS listing_id,
      date_date,                         
      available_bool,
      is_occupied,
      price_numeric,
      adjusted_price_numeric,            
      minimum_nights,
      maximum_nights
    FROM mvp_airbnb_catalog.default.silver_calendar_nyc
),
joined AS (
    SELECT
      c.city,
      c.listing_id,
      c.date_date,
      c.available_bool,
      c.is_occupied,
      c.price_numeric,
      c.adjusted_price_numeric,
      c.minimum_nights,
      c.maximum_nights,
      dl.price_numeric       AS listing_price_numeric,
      dl.has_valid_price
    FROM calendar_union c
    INNER JOIN mvp_airbnb_catalog.default.dim_listings dl
      ON dl.city       = c.city
     AND dl.listing_id = c.listing_id
)
SELECT
  city,
  listing_id,
  date_date,
  available_bool,
  is_occupied,
  price_numeric,
  adjusted_price_numeric,
  minimum_nights,
  maximum_nights,
  CASE
    WHEN is_occupied = TRUE AND has_valid_price = TRUE
      THEN listing_price_numeric
    ELSE 0
  END AS revenue_estimated
FROM joined
;

-- =====================================================================
-- FACT_REVIEWS  (Gold a partir das Silver de reviews + dim_listings)
-- =====================================================================

CREATE OR REPLACE TABLE mvp_airbnb_catalog.default.fact_reviews AS
WITH reviews_union AS (
    SELECT
      'Rio de Janeiro'  AS city,
      *
    FROM mvp_airbnb_catalog.default.silver_reviews_rio

    UNION ALL

    SELECT
      'New York City'   AS city,
      *
    FROM mvp_airbnb_catalog.default.silver_reviews_nyc
),
joined AS (
    SELECT
      r.city,
      CAST(r.listing_id AS BIGINT)         AS listing_id,
      CAST(r.id AS BIGINT)                 AS review_id,      
      r.review_date,                                           
      r.comments_clean
    FROM reviews_union r
    INNER JOIN mvp_airbnb_catalog.default.dim_listings dl
      ON dl.city = r.city
     AND dl.listing_id = CAST(r.listing_id AS BIGINT)
)
SELECT
  city,
  listing_id,
  review_id,
  review_date,
  comments_clean
FROM joined
;

-- PK: (city, review_id)
ALTER TABLE mvp_airbnb_catalog.default.fact_reviews
ALTER COLUMN city      SET NOT NULL;

ALTER TABLE mvp_airbnb_catalog.default.fact_reviews
ALTER COLUMN review_id SET NOT NULL;

ALTER TABLE mvp_airbnb_catalog.default.fact_reviews
ADD CONSTRAINT pk_fact_reviews
PRIMARY KEY (city, review_id)
;

-- FK: fact_reviews.(city, listing_id) -> dim_listings
ALTER TABLE mvp_airbnb_catalog.default.fact_reviews
ADD CONSTRAINT fk_fact_reviews_listing
FOREIGN KEY (city, listing_id)
REFERENCES mvp_airbnb_catalog.default.dim_listings (city, listing_id)
;


In [0]:
%sql
-- Evidência dim_listings
SELECT * FROM mvp_airbnb_catalog.default.dim_listings
LIMIT 5

In [0]:
%sql
-- Evidência dim_hosts
SELECT * FROM mvp_airbnb_catalog.default.dim_hosts
LIMIT 5

In [0]:
%sql
-- Evidência dim_neighbourhoods
SELECT * FROM mvp_airbnb_catalog.default.dim_neighbourhoods
LIMIT 5

In [0]:
%sql
-- Evidência fact_reviews
SELECT * FROM mvp_airbnb_catalog.default.fact_reviews
LIMIT 5

In [0]:
%sql
-- Evidência fact_calendar
SELECT * FROM mvp_airbnb_catalog.default.fact_calendar
LIMIT 5